In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(executable_path='C:/Users/Administrator/Desktop/chromedriver_win32/chromedriver.exe', options=chrome_options)
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [4]:
# https://www.boolaw.com/chosen/03c51fe2.html
def getMp3UrlAndText(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    temp = tree.xpath('//div[@class="move fl"]/audio/@src')
    if len(temp) == 0:
        #time.sleep(60)
        return '', ''
    mp3Url = temp[0]
    allText = tree.xpath('//div[@class="voice-det-con"]//text()')
    text = '\n'.join(allText)
    return mp3Url, text

In [5]:
def downloadAudioAndText(url, fileName):
    try:
        mp3Url, text = getMp3UrlAndText(url)
        if mp3Url == '':
            print(fileName, ' empty')
            return
        with open('text/' + fileName + '.txt', 'w', encoding='utf-8') as fp:
            fp.write(text)
            print(fileName, ' text successful')
        content = getResponse(mp3Url).content
        with open('mp3/' + fileName + '.mp3', 'wb') as fp:
            fp.write(content)
            print(fileName, ' mp3 successful')
    except Exception as e:
        print(fileName, 'error')

In [13]:
downloadAudioAndText('https://www.boolaw.com/chosen/8057004b.html', 'test3')

test3  text successful
test3  mp3 successful


In [6]:
def getAllPageUrl(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    liList = tree.xpath('//div[@class="switch-item"]/ul/li')
    linkList = []
    for li in liList:
        pageUrl = 'https:' + li.xpath('./h2/a/@href')[0]
        #print(pageUrl)
        linkList.append(pageUrl)
    return linkList

In [7]:
def getAllPageUrl(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    divList = tree.xpath('//div[@class="lawnoz"]/div')
    linkList = []
    for div in divList:
        pageUrl = 'https://www.boolaw.com' + div.xpath('./h6/a/@href')[0]
        #print(pageUrl)
        linkList.append(pageUrl)
    return linkList

In [14]:
getAllPageUrl('https://www.boolaw.com/knows/_1/')

https://www.boolaw.com/chosen/8057004b.html
https://www.boolaw.com/chosen/7f395ae9.html
https://www.boolaw.com/chosen/7e49f551.html
https://www.boolaw.com/chosen/7aa9ce2e.html
https://www.boolaw.com/chosen/79f2d7b0.html
https://www.boolaw.com/chosen/79abfba1.html
https://www.boolaw.com/chosen/77dcf7c3.html
https://www.boolaw.com/chosen/77683eb7.html
https://www.boolaw.com/chosen/2131d3c5.html
https://www.boolaw.com/chosen/e61507d3.html


['https://www.boolaw.com/chosen/8057004b.html',
 'https://www.boolaw.com/chosen/7f395ae9.html',
 'https://www.boolaw.com/chosen/7e49f551.html',
 'https://www.boolaw.com/chosen/7aa9ce2e.html',
 'https://www.boolaw.com/chosen/79f2d7b0.html',
 'https://www.boolaw.com/chosen/79abfba1.html',
 'https://www.boolaw.com/chosen/77dcf7c3.html',
 'https://www.boolaw.com/chosen/77683eb7.html',
 'https://www.boolaw.com/chosen/2131d3c5.html',
 'https://www.boolaw.com/chosen/e61507d3.html']

In [8]:
# https://www.boolaw.com/knows/_10/
total = 1
for i in range(1, 11):
    url = f'https://www.boolaw.com/knows/_{i}/'
    print(url)
    for link in getAllPageUrl(url):
        print(link)
        downloadAudioAndText(link, str(total))
        total = total + 1

https://www.boolaw.com/knows/_1/
https://www.boolaw.com/chosen/8057004b.html
1  text successful
1  mp3 successful
https://www.boolaw.com/chosen/7f395ae9.html
2  empty
https://www.boolaw.com/chosen/7e49f551.html
3  text successful
3  mp3 successful
https://www.boolaw.com/chosen/7aa9ce2e.html
4  text successful
4  mp3 successful
https://www.boolaw.com/chosen/79f2d7b0.html
5  text successful
5  mp3 successful
https://www.boolaw.com/chosen/79abfba1.html
6  text successful
6  mp3 successful
https://www.boolaw.com/chosen/77dcf7c3.html
7  text successful
7  mp3 successful
https://www.boolaw.com/chosen/77683eb7.html
8  text successful
8  mp3 successful
https://www.boolaw.com/chosen/2131d3c5.html
9  text successful
9  mp3 successful
https://www.boolaw.com/chosen/e61507d3.html
10  text successful
10  mp3 successful
https://www.boolaw.com/knows/_2/
https://www.boolaw.com/chosen/e57cbf7e.html
11  text successful
11  mp3 successful
https://www.boolaw.com/chosen/e409c41b.html
12  text successful
12 